# Algorithmic Motion Planning (236610)
John Noonan and Eli Shafer<br>
Homework 1<br>
November 2019

*QUESTION 1*: *Given sets A, B, and C formally* **Prove** *that $A \oplus (B \cup C) = (A \oplus B) \cup (A \oplus C)$*

**Proof**:

Let $p \in A \oplus (B \cup C)$.  

Then $\exists q \in A$ and $\exists r \in (B \cup C)$ s.t. $p = q + r$ (according to the definition of the Minkowski sum). 

If $r \in B$, then $A \oplus (B \cup C) = A \oplus B$.  

Or if $r \in C$, then $A \oplus (B \cup C) = A \oplus C$.  

Thus, $A \oplus (B \cup C) = (A \oplus B) \cup (A \oplus C)$. 
<p style='text-align: right;'> $\blacksquare$ </p>

*QUESTION 2*: **What** is the Minkowski sum (what geometry object and what can you say about it) of (i) Two points? (ii) A point and a line? (iii) Two lines (think of all possible cases)? (iv) Two Disks?

**Answer 2**:

i) The Minkowski sum of two points is another point. If $P = \{p_1\}$ and $Q = \{q_1\}$ for $p_1, q_1 \in \mathbb{R}^d$ then $P \oplus Q$ = $\{p_1+q_1\}$ where $(p_1 + q_1) \in \mathbb{R}^d$.

ii) The Minkowski sum of a point and a line is the line translated by the point.  

This can be shown via parameterization. Let $\mathbf{p}$ and $\mathbf{q}$ be vector functions defined as: $\mathbf{p}(t) = p + t\mathbf{v}$ where $t \in \mathbb{R}$ and $p,\mathbf{v} \in \mathbb{R}^d$ and $\mathbf{q}(r) = q + r\mathbf{w}$ where $r \in \mathbb{R}$ and $q,\mathbf{w} \in \mathbb{R}^d$.  Then the Minkowski sum of a line and point can be parametrically represented by vector functions $\mathbf{p}$ and $\mathbf{q}$ such that $\mathbf{p}(t) \oplus \mathbf{q}(0) = p + q + t\mathbf{v} = (p + q) + t\mathbf{v}$, thus corresponding to the line translated by the point.

iii) If the lines are parallel, then the Minkowski sum is a line with the same direction (slope).  If we consider line *segments* then the Minkowski sum of the parallel lines will be a line longer in length but again with the same direction (slope). If the lines are perpendicular, then the Minkowski sum is a square.  And if the lines are non-parallel, then the Minkowski sum is a convex polygon.

iv) The Minkowski sum of two discs is a disc.  If each disc is defined by a center point and a radius, then the Minkowski sum is a disc whose center point is the sum of the two discs' center points and whose radius is the sum of the two discs' radii.

*QUESTION 3*: Recall that for the proof of Lemma. 6.2 (complexity of a trapezoidal map) we used the property that in a planar graph we have that $$E \leq 3V - 6$$
Here E and V are the number of edges and vertices in a planar graph, respectively. **Prove** Eq. 1.


**Answer 3**:

Let F denote the number of faces in the planar graph.  Each edge in the planar graph is adjacent to two faces, and each face has at least 3 edges adjacent to it.  Thus, $$2E \geq 3F$$  Using Euler's formula which states that $V - E + F = 2$, we conclude the following:

$$2E \geq 3F$$
$$2E \geq 3(2 - V + E)$$
$$2E \geq 6 - 3V + 3E$$
$$-E \geq 6 - 3V$$
$$ E \leq 3V - 6$$ 

<p style='text-align: right;'> $\blacksquare$ </p>